In [1]:
import os
import scipy
import librosa
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from skimage.transform import resize
from IPython.display import clear_output
from matplotlib.ticker import MaxNLocator

2023-11-05 14:05:04.517475: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-05 14:05:05.061576: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-05 14:05:05.061620: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-05 14:05:05.064311: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-05 14:05:05.364284: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-05 14:05:05.366472: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [3]:
# our usual useful functions

class PlotLossAccuracy(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.acc = []
        self.losses = []
        self.val_losses = []
        self.val_acc = []
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):

        self.logs.append(logs)
        self.x.append(int(self.i))
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('accuracy'))
        self.val_acc.append(logs.get('val_accuracy'))

        self.i += 1

        clear_output(wait=True)
        plt.figure(figsize=(16, 6))
        plt.plot([1, 2])
        plt.subplot(121)
        plt.plot(self.x, self.losses, label="train loss")
        plt.plot(self.x, self.val_losses, label="validation loss")
        plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.title('Model Loss')
        plt.legend()
        plt.subplot(122)
        plt.plot(self.x, self.acc, label="training accuracy")
        plt.plot(self.x, self.val_acc, label="validation accuracy")
        plt.legend()
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.title('Model Accuracy')
        plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))
        plt.show();


def save_model_to_disk(model, filename_base):
    # save model and weights (don't change the filenames)
    model_json = model.to_json()
    with open(filename_base + ".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(f"{filename_base}.h5")
    print(f"Saved model to {filename_base}.json and weights to {filename_base}.h5")

In [2]:
x_d = np.load('xtrain.npy')
y_num = np.load('ytrain_num.npy')
y_gender = np.load('ytrain_gender.npy')

In [3]:
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score


def prepare_data(X,Y):
    X_train, X_validation, Y_train, Y_validation = sklearn.model_selection.train_test_split(X, Y, test_size=.1)
    one_hot_encoder = OneHotEncoder(sparse_output=False)
    Y_train = one_hot_encoder.fit_transform(Y_train.reshape(-1, 1))
    Y_validation = one_hot_encoder.fit_transform(Y_validation.reshape(-1, 1))
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1, 1)
    X_validation = X_validation.reshape(X_validation.shape[0], X_validation.shape[1], 1, 1)
    return X_train, X_validation, Y_train, Y_validation

X_train, X_validation, Y_train, Y_validation = prepare_data(x_d, y_gender)



In [13]:
def spectrum_data(audio_tracks):
    X = np.empty((25000,227, 128))
    for audio_track in audio_tracks:
        f, t, Zxx = scipy.signal.stft(audio_track, 8000, nperseg=455, noverlap=393, window='hann')
        Zxx_ = np.abs(Zxx[0:227, 2:-1])
        Zxx_ = np.atleast_3d(Zxx_).transpose(2,0,1)
        # after the STFT, the spectrogram is simply computed by taking the amplitude of the Fourier coefficients.
        Zxx_ = librosa.amplitude_to_db(Zxx_, ref=np.max)
        np.append(X, Zxx_)
        #X.append(Zxx_)
    return X

new_X = spectrum_data(x_d)
print(new_X.shape)
print(x_d.shape)


**Exercise 1A**

In [8]:
from keras.layers import Dense, Dropout, Flatten, Normalization, Conv1D, MaxPool1D

inputs = keras.layers.Input(shape=(8000,1))
x = inputs

x = Conv1D(16, 3, activation='relu', strides=(1))(x)
x = Conv1D(16, 3, activation='relu', strides=(1))(x)
x = Conv1D(16, 3, activation='relu', strides=(1))(x)
x = MaxPool1D(4, name='second_layer')(x)
x = Dropout(0.25)(x)

x = Conv1D(8, 3, activation='relu', strides=(1))(x)
x = Conv1D(8, 3, activation='relu', strides=(1))(x)
x = Conv1D(8, 3, activation='relu', strides=(1))(x)
x = MaxPool1D(2, name='end_layer')(x)
x = Dropout(0.25)(x)

x = Flatten()(x)

x = Dropout(0.25)(x)
x = Dense(6, activation='relu')(x)

predictions = Dense(2, activation='sigmoid')(x)

model = keras.models.Model(inputs=inputs, outputs=predictions)
opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt,
                    loss='binary_crossentropy',
                    metrics=['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 8000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 7998, 16)          64        
                                                                 
 conv1d_1 (Conv1D)           (None, 7996, 16)          784       
                                                                 
 conv1d_2 (Conv1D)           (None, 7994, 16)          784       
                                                                 
 second_layer (MaxPooling1D  (None, 1998, 16)          0         
 )                                                               
                                                                 
 dropout_1 (Dropout)         (None, 1998, 16)          0         
                                                           

**Exercise 1B**

In [63]:
from keras.layers import Dense, Dropout, Flatten, Normalization, Conv1D, MaxPool1D

inputs = keras.layers.Input(shape=(8000,1))
x = inputs  # Don't flatten until after your 2D layers
print(x.shape)
x = Conv1D(64, 3, activation='relu', strides=(1))(x)
x = Conv1D(64, 3, activation='relu', strides=(1))(x)
x = Conv1D(64, 3, activation='relu', strides=(1))(x)
x = MaxPool1D(4, name='first_layer')(x)
  #x = Dropout(0.25)(x)
x = Dropout(0.25)(x)


x = Conv1D(32, 3, activation='relu', strides=(1))(x)
x = Conv1D(32, 3, activation='relu', strides=(1))(x)
x = Conv1D(32, 3, activation='relu', strides=(1))(x)
x = MaxPool1D(4, name='second_layer')(x)
x = Dropout(0.25)(x)


x = Conv1D(8, 3, activation='relu', strides=(1))(x)
x = Conv1D(8, 3, activation='relu', strides=(1))(x)
x = Conv1D(8, 3, activation='relu', strides=(1))(x)
x = MaxPool1D(4, name='end_layer')(x)
x = Dropout(0.25)(x)

x = Flatten()(x)
print(x.shape)
#x = Dense(512, activation='relu')(x)
x = Dense(24, activation='relu')(x)
#x = Dropout(0.5)(x)
x = Dropout(0.3)(x)
#x = Dense(128, activation='relu')(x)
x = Dense(16, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(16, activation='relu')(x)


predictions = Dense(10, activation='softmax')(x)

model = keras.models.Model(inputs=inputs, outputs=predictions)
opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt,
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
model.summary()

(None, 8000, 1)
(None, 7968)
Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 8000, 1)]         0         
                                                                 
 conv1d_1 (Conv1D)           (None, 7998, 32)          128       
                                                                 
 conv1d_2 (Conv1D)           (None, 7996, 32)          3104      
                                                                 
 conv1d_3 (Conv1D)           (None, 7994, 32)          3104      
                                                                 
 first_layer (MaxPooling1D)  (None, 1998, 32)          0         
                                                                 
 dropout_32 (Dropout)        (None, 1998, 32)          0         
                                                                 
 conv1d_4 (Conv1D)           

**LSTM**

In [9]:
from keras.layers import Dense, Dropout, Flatten, Normalization, Conv1D, MaxPool1D, LSTM

inputs = keras.layers.Input(shape=(8000,1))
x = inputs

x = LSTM(64, return_sequences=True)(inputs)
x = LSTM(32)(x)

x = Flatten()(x)

x = Dropout(0.25)(x)
x = Dense(6, activation='relu')(x)

predictions = Dense(2, activation='sigmoid')(x)

model = keras.models.Model(inputs=inputs, outputs=predictions)
opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt,
                    loss='binary_crossentropy',
                    metrics=['accuracy'])
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 8000, 1)]         0         
                                                                 
 lstm_2 (LSTM)               (None, 8000, 64)          16896     
                                                                 
 lstm_3 (LSTM)               (None, 32)                12416     
                                                                 
 flatten_2 (Flatten)         (None, 32)                0         
                                                                 
 dropout_4 (Dropout)         (None, 32)                0         
                                                                 
 dense_4 (Dense)             (None, 6)                 198       
                                                                 
 dense_5 (Dense)             (None, 2)                 14  

In [7]:
num_epochs = 300
pltCallBack = PlotLossAccuracy()
model.fit(X_train, Y_train,
        batch_size=32, epochs=num_epochs,
        validation_data=(X_validation, Y_validation),
        callbacks=[pltCallBack])


Epoch 1/300
  2/704 [..............................] - ETA: 2:49:54 - loss: 0.6930 - accuracy: 0.7188

KeyboardInterrupt: 

In [ ]:
model_1A = model  # Figure this out
save_model_to_disk(model_1A, 'waveform_gender_model')

In [ ]:
model_1B = # Figure this out
save_model_to_disk(model_1B, 'waveform_digit_model')

In [ ]:
model_2A = # Figure this out
save_model_to_disk(model_2A, 'spectrogram_gender_model')

In [ ]:
model_2B = # Figure this out
save_model_to_disk(model_2B, 'spectrogram_digit_model')